<a href="https://colab.research.google.com/github/Heisnotanimposter/ObjectDetection_with_Server/blob/main/PersonSpeedvision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!nvidia-smi

Wed Aug 21 14:26:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
!pip install -q supervision ultralytics
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.1/869.1 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.5 MB/s eta 0:00:00


In [7]:
import cv2
import numpy as np
import time
from ultralytics import YOLO
import supervision as sv
from collections import defaultdict, deque

# Load the YOLO model
model = YOLO("yolov8s.pt")  # Use the appropriate YOLOv8 model variant

# Set up video paths
SOURCE_VIDEO_PATH = "/content/drive/MyDrive/Team7dataset/Team7Shared/140sNightShinjuku.mp4"
# Initialize video capture
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)

# Initialize ByteTrack tracker
byte_track = sv.ByteTrack(
    track_activation_threshold=0.3,
    lost_track_buffer=30,
    frame_rate=cap.get(cv2.CAP_PROP_FPS)
)

# Initialize data structures to store past positions for speed estimation
past_positions = defaultdict(lambda: deque(maxlen=5))

# Initialize LED state and timer
led_signal = 0  # 0 means no person detected, 1 means person detected
led_state = "off"
last_led_change_time = time.time()
cycle_duration = 30  # 30 seconds cycle (15s green, 15s red)

# Process the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO object detection
    results = model(frame, conf=0.5)  # Adjust confidence threshold as needed

    # Get detections
    detections = sv.Detections.from_ultralytics(results[0])

    # Filter detections by confidence
    detections = detections[detections.confidence > 0.3]

    # Update tracker with detections
    tracks = byte_track.update_with_detections(detections)

    # Check if any person is detected
    person_detected = any(track[4] == 0 for track in tracks)  # Assuming class_id is the 5th element in the tuple

    if person_detected:
        led_signal = 1
    else:
        led_signal = 0

    # LED light control logic
    current_time = time.time()
    elapsed_time = current_time - last_led_change_time

    if elapsed_time >= cycle_duration / 2:
        # Toggle LED state
        last_led_change_time = current_time
        if led_signal == 1:  # If a person is detected, the green light turns on
            led_state = "green" if led_state == "off" else "off"
        else:  # If no person is detected, the red light turns on
            led_state = "red" if led_state == "off" else "off"

    # Simulate LED light output (print to console)
    print(f"LED State: {led_state}")

    # Optional: Annotate the frame and display it
    for track in tracks:
        track_id = track[0]  # Track ID
        bbox = track[1:5]  # Bounding box coordinates
        class_id = track[4]  # Class ID

        if class_id == 0:  # Only annotate persons (class_id 0)
            x1, y1, x2, y2 = map(int, bbox)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'ID: {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()



0: 384x640 1 person, 12.2ms
Speed: 1.9ms preprocess, 12.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
LED State: off


DisabledFunctionError: cv2.imshow() is disabled in Colab, because it causes Jupyter sessions
to crash; see https://github.com/jupyter/notebook/issues/3935.
As a substitution, consider using
  from google.colab.patches import cv2_imshow
